In [36]:
%run ./Definitions.ipynb

2020-01-03 22:10:47.531001
thunder  Fri Jan  3 22:10:48 2020
[0] GeForce GTX 1080 Ti | 58'C,  57 % | 10936 / 11178 MB | david(10926M)
[1] GeForce GTX 1080 Ti | 54'C,  68 % | 10936 / 11178 MB | david(10926M)
[2] GeForce GTX 1080 Ti | 29'C,   0 % |    10 / 11178 MB |
[3] GeForce GTX 1080 Ti | 63'C,   1 % | 10936 / 11178 MB | david(10926M)
[4] GeForce GTX 1080 Ti | 53'C,   0 % | 10936 / 11178 MB | david(10926M)
[5] GeForce GTX 1080 Ti | 28'C,   0 % |    10 / 11178 MB |
[6] GeForce GTX 1080 Ti | 31'C,   0 % |    10 / 11178 MB |
[7] GeForce GTX 1080 Ti | 29'C,   0 % |    10 / 11178 MB |
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Dataset consist of 85730 patients data with 6251 features for each patient
Function definitions executed
Definitions DONE !!



In [2]:
!nvidia-smi
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # so the IDs match nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"] = "5"   # choose here your GPU (0-7)

import tensorflow as tf
import tensorflow.compat.v1 as tf
import keras
# from keras import backend as K
import tensorflow.python.keras.backend as K

Fri Jan  3 19:59:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.01    Driver Version: 418.87.01    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 46%   62C    P2   109W / 250W |  10936MiB / 11178MiB |     79%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:06:00.0 Off |                  N/A |
| 40%   55C    P2    74W / 250W |  10936MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

https://www.pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/

## Loading numerical and categorical data

In [5]:
# LOAD
meta_train = pd.read_pickle(tts_path + '40_features_union_train_df_FINAL.pkl')
meta_val = pd.read_pickle(tts_path + '40_features_union_val_df_FINAL.pkl')
meta_train.shape, meta_val.shape

((51438, 321), (17146, 321))

## Loading Ground truth

In [6]:
# LOAD
y_train = pd.read_pickle(tts_path + 'yu_train.pkl')
y_val = pd.read_pickle(tts_path + 'yu_val.pkl')
y_train.shape, y_val.shape

((51438, 1), (17146, 1))

## Loading Fundus Image data

In [11]:
# INITIALIZATION

img_size = 250
red_size = 224

srcpath = '/ds2/data/retina/UK_BB/UK_BB_DATA/fundus/'
funduspath_CR = DATA_dir + 'Fundus_images_250_CR/'
trainpath = funduspath_CR + 'train/'
valpath = funduspath_CR + 'val/'
testpath = funduspath_CR + 'test/'

In [12]:
#AUGMENTATION

seq = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Affine(scale=1.1),
    iaa.Affine(rotate=(-10, 10)),
    # iaa.GammaContrast(0.9),
    iaa.Resize({"height": 230, "width": 230}),
    iaa.CropToFixedSize(width=red_size, height=red_size)
    ])

In [84]:
img_train = pd.read_pickle(funduspath_CR + 'valid_traindf_names.pkl')
img_val = pd.read_pickle(funduspath_CR + 'valid_valdf_names.pkl')

In [86]:
img_train.shape, img_val.shape

((101834, 5), (33943, 5))

In [165]:
img_train['GTu'] = img_train['GTu'].astype(int)
img_val['GTu'] = img_val['GTu'].astype(int)

# Model

2 ways to combine both inputs
https://stats.stackexchange.com/questions/410745/combining-random-forests-and-neural-networks

In [81]:
def create_mlp(dim):
    # define our MLP network
    model = Sequential()
    model.add(Dense(64, input_dim=dim, activation="relu"))
    model.add(Dense(4, activation="relu"))

    # return our model
    return model

def create_cnn():       
    mobile_conv = MobileNet(weights='imagenet', include_top=False, input_shape=(red_size, red_size, 3))

    # UnFreeze last conv layer
    for layer in mobile_conv.layers[:78]:
        layer.trainable = False
    for layer in mobile_conv.layers[78:]:
        layer.trainable = True
        
    # Create the model
    model = models.Sequential()
    model.add(mobile_conv)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(4, activation='relu'))
 
    return model

learning_rate  = 0.0001 
decay = 1e-5
opt = Adam(lr=learning_rate, decay = decay) 
BATCH_SIZE = 32
METRICS = [keras.metrics.BinaryAccuracy(name='Accuracy'),
           Precision, 
           Recall, 
           F1,
           keras.metrics.AUC(name='AUC')]

In [73]:
# create the MLP and CNN models
mlp = create_mlp(meta_train.shape[1]) # categorical data
cnn = create_cnn() # left eye input

combinedInput = concatenate([mlp.output, cnn.output])
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=[mlp.input, cnn.input], outputs=x)


In [75]:
model.compile(loss='binary_crossentropy', metrics=METRICS, optimizer=opt)
# model.summary()

In [198]:
# train the model
n_epochs = 2
print("[INFO] training model...")
hist = model.fit_generator(train_generator, 
                              epochs=n_epochs, 
                              steps_per_epoch=5,
                              validation_data=validation_generator,
                              validation_steps=10)

[INFO] training model...
Epoch 1/2


StopIteration: 

In [187]:
def get_img(path):
    img = imageio.imread(path)
    return(img)

def preprocess_input(image):
    rescale= 1./127.5
    image = image * rescale
    image = image - 1
    images = [image, image, image, image]
    images_aug = seq.augment_images(images)
    return(images_aug[1])

In [190]:
train_generator = custom_generator(meta_train, y_train, img_train, trainpath, batch_size = 32, aug=True)
validation_generator = custom_generator(meta_val, y_val, img_val, valpath, batch_size = 32, aug=False)

In [192]:
def custom_generator(X1,Y1,X2,df_path,batch_size = 32,aug=True):
    
    indices = np.arange(len(X1))
    
    while True:
        samples = img_train.sample(batch_size)
        imgs = samples[samples.columns[0]]
        labels = samples[samples.columns[4]]
        f_indices = np.random.choice(indices,  size = batch_size)
        
        img_input = []
        meta_input = []
        meta_label = []
          
        # Read in each input, and perform preprocessing
        
        for i in imgs:
            inp = get_img(df_path+i)
            if (aug == True):
                inp = preprocess_input(image=inp)
            img_input.append(inp)
                
        for i in f_indices:
            feat = X1.iloc[i].tolist()
            f_label = Y1.iloc[i]
            
            meta_input.append(feat)
            meta_label.append(f_label)
            
        batch_img = np.array( img_input )
        img_label = np.array( labels )
        
        batch_feature = np.array( meta_input )
        feat_label = np.array( meta_label )

        yield( batch_img,img_label, batch_feature,feat_label )

In [189]:
custom_generator(meta_train, y_train, img_train, trainpath, batch_size = 32, aug=True)

<generator object custom_generator at 0x7fd197713a50>

In [ ]:
 
# create the input to our final set of layers as the *output* of both
# the MLP and CNN
combinedInput = concatenate([mlp.output, cnn.output])
 
# our final FC layer head will have two dense layers, the final one
# being our regression head
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)
 
# our final model will accept categorical/numerical data on the MLP
# input and images on the CNN input, outputting a single value (the
# predicted price of the house)
model = Model(inputs=[mlp.input, cnn.input], outputs=x)

## Random Forest Classifier

## Models

In [9]:
def create_mlp(dim, regress=False):
    # define our MLP network
    model = Sequential()
    model.add(Dense(8, input_dim=dim, activation="relu"))
    model.add(Dense(4, activation="relu"))

    # check to see if the regression node should be added
    if regress:
        model.add(Dense(1, activation="linear"))

    # return our model
    return model

In [10]:
def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False):
    # initialize the input shape and channel dimension
    inputShape = (height, width, depth)
    chanDim = -1

    # define the model input
    inputs = Input(shape=inputShape)

    # loop over the number of filters
    for (i, f) in enumerate(filters):
        # if this is the first CONV layer then set the input
        # appropriately
        if i == 0:
            x = inputs

        # CONV => RELU => BN => POOL
        x = Conv2D(f, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)

    # flatten the volume, then FC => RELU => BN => DROPOUT
    x = Flatten()(x)
    x = Dense(16)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Dropout(0.5)(x)

    # apply another FC layer, this one to match the number of nodes
    # coming out of the MLP
    x = Dense(4)(x)
    x = Activation("relu")(x)

    # check to see if the regression node should be added
    if regress:
        x = Dense(1, activation="linear")(x)

    # construct the CNN
    model = Model(inputs, x)

    # return the CNN
    return model

## Split image data as per above train_x and val_x

In [12]:
## OLD
# train_x, val_x, train_y, val_y = train_test_split(df, y, test_size = 0.1, random_state = 13, stratify=y)
# print(train_x.shape, val_x.shape, train_y.shape, val_y.shape)

(75699, 15) (8411, 15) (75699,) (8411,)


In [ ]:
# partition the data into training and testing splits using 75% of the data for training and the remaining 25% for testing
print("[INFO] processing data...")
(trainAttrX, testAttrX, trainImagesX, testImagesX) = train_test_split(data_df, images, test_size=0.25, random_state=42)
 
# find the largest house price in the training set and use it to
# scale our house prices to the range [0, 1] (will lead to better
# training and convergence)
maxPrice = trainAttrX["price"].max()
trainY = trainAttrX["price"] / maxPrice
testY = testAttrX["price"] / maxPrice
 
# process the house attributes data by performing min-max scaling
# on continuous features, one-hot encoding on categorical features,
# and then finally concatenating them together
(trainAttrX, testAttrX) = datasets.process_house_attributes(df,
	trainAttrX, testAttrX)